## step1. import all required modules and packages

In [1]:
from sklearn.impute import KNNImputer
import numpy as np
import pandas as pd
import csv
import random
import matplotlib.pyplot as plt
from scipy.spatial import distance
from sklearn.metrics.pairwise import nan_euclidean_distances
# dist(x,y) = sqrt(weight * sq. distance from present coordinates) where, weight = Total # of coordinates / # of present coordinates
# ref : https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.nan_euclidean_distances.html
import numpy.ma as ma
##configure precision for floating numbers
np.set_printoptions(precision=3)

### define all functions section

In [2]:
def dominate(v1, v2):
    
    dom = False
    for d in range(len(v1)):        
        if(d == 1):
            if(v1[d] <= v2[d]):
                dom = True
            else:
                dom = False
                break
        elif(d == 0 or d == 2 or d == 3):
            if(v1[d] >= v2[d]):
                dom = True
            else:
                dom = False
                break
    
    return dom
    

def findskyline(array):
    rows, cols = array.shape
#     print("row of array = ", rows)
#     print("col of array = ", cols)
    
    skylineList = [0]
    candidate = []
    for row in range(rows):
#         print("iteration : ", row)
        new = array[row]
        length = len(skylineList)
        
        for index in range(length):
            temp = skylineList.pop(0)
#             print("pop out element : "+str(temp))
            pop = array[temp]
            
            if(not(dominate(new, pop)) and not(dominate(pop, new))):
#                 print(str(new)+ " and "+ str(pop) + " don't dominate each other.")
                if(index == (length-1) ):
#                     print("last index:" + str(index))
#                     print("insert " + str(row) + " into candidate." )    
                    if(row not in candidate):
                        candidate.append(row)
                
#                 print("insert " + str(temp) + " into candidate." )    
                if(temp not in candidate):
                    candidate.append(temp)
            
            elif(dominate(new, pop)):
#                 print(str(new) + " dominate " + str(pop))
#                 print("insert " + str(row) + " into candidate." )
                if(row not in candidate):
                    candidate.append(row)
            
            elif(dominate(pop, new)):
#                 print(str(pop) + " dominate " + str(new))
#                 print("insert " + str(temp) + " into candidate." )
                if(temp not in candidate):
                    candidate.append(temp)
            
#             print("candidate : "+str(candidate))
        
        skylineList = candidate[:]
        candidate.clear()
            
#         print("current skylineList:"+str(skylineList))

    return skylineList

def eval_missing_weight(dataset):
    
    data = dataset.copy()
    rows, cols = dataset.shape
    
    #initialize a weight list with length same with number of columns of input dataset
    missing_weight_by_each_col = [None]*cols 
    miss_count = [None]*cols
    miss_counter_per_col = 0
    
    for col in range(cols):
        for row in range(rows):
            if np.isnan(data[row][col]):
                miss_counter_per_col += 1
            else:
                continue
        
        miss_count[col] = miss_counter_per_col #record missing count for each column
        missing_weight_by_each_col[col] = miss_counter_per_col/rows #record missing ratio for each column
        miss_counter_per_col = 0 # reset the missing-counter into next iteration
    
    print(miss_count)
    print(missing_weight_by_each_col)
    return miss_count, missing_weight_by_each_col

## step2. prepare for input data with some missing values

In [3]:
'''
    Incomplete data set
'''
data  = np.array([
    [np.nan,      5,      3, np.nan,      6],
    [2     , np.nan,      6, np.nan, np.nan],
    [8     ,      9, np.nan, np.nan, np.nan],
    [np.nan,      8,      6,      7, np.nan],
    [5     ,      6, np.nan, np.nan, np.nan],
    [9     ,      5, np.nan,      7, np.nan],
    ])

miss_count, miss_weight = eval_missing_weight (data)
data_row, data_col = data.shape

original_skyline = findskyline(data) 
print(original_skyline)

[2, 1, 3, 4, 5]
[0.3333333333333333, 0.16666666666666666, 0.5, 0.6666666666666666, 0.8333333333333334]
[1, 2, 3, 4, 5, 0]


## step3. evaluate the distance_matrix and weight_matrix

In [4]:
'''
    distance_matrix
'''
distance_matrix = nan_euclidean_distances(data,data)
dis_row, dis_col = distance_matrix.shape 
''' there are two cases of distance_matrix = 0
    1. two samples have same value but non-NaN with its all attributes
    2. two samples have no any corresponding attributes both are non-NaN
'''
print(distance_matrix)

'''
    weight_matrix
'''
# initialize weight_matrix of same size with distance_matrix
weight_matrix = [ [ None for y in range( dis_col ) ] for x in range( dis_row ) ]

# print(weight_matrix)

# assign values according to value in distance_matrix
for row in range(dis_row):
    for col in range(dis_col):
        if (row == col or distance_matrix[row][col] == 0): 
            # assign zero otherwise (index i = j)
            '''
                there are two cases causing weight_matrix = 0
                1. weight_matrix row index = column index
                2. distance_matrix value is zero, which means neigther corresponding attributes of two samples are non-NaN
            '''
            weight_matrix[row][col] = 0
        elif (row != col and distance_matrix[row][col] != 0) :
            # assign inverse value of nan_euclidean_distances w.r.t distance_matrix
            # python will alert divide-by-zero warning here
            weight_matrix[row][col] = 1. / distance_matrix[row][col]

# print(weight_matrix)
weight_matrix

[[ 0.     6.708  8.944  6.708  2.236  0.   ]
 [ 6.708  0.    13.416  0.     6.708 15.652]
 [ 8.944 13.416  0.     2.236  6.708  6.519]
 [ 6.708  0.     2.236  0.     4.472  4.743]
 [ 2.236  6.708  6.708  4.472  0.     6.519]
 [ 0.    15.652  6.519  4.743  6.519  0.   ]]


[[0,
  0.14907119849998599,
  0.11180339887498948,
  0.14907119849998599,
  0.4472135954999579,
  0],
 [0.14907119849998599,
  0,
  0.07453559924999299,
  0,
  0.14907119849998599,
  0.06388765649999399],
 [0.11180339887498948,
  0.07453559924999299,
  0,
  0.4472135954999579,
  0.14907119849998599,
  0.15339299776947407],
 [0.14907119849998599,
  0,
  0.4472135954999579,
  0,
  0.22360679774997896,
  0.21081851067789195],
 [0.4472135954999579,
  0.14907119849998599,
  0.14907119849998599,
  0.22360679774997896,
  0,
  0.15339299776947407],
 [0,
  0.06388765649999399,
  0.15339299776947407,
  0.21081851067789195,
  0.15339299776947407,
  0]]

In [5]:
'''
sorting the indices each row of distance_matrix
'''
def sorted_index_per_row(one_row_dis_matrix, row):
    s = one_row_dis_matrix
    sorted_s = sorted(range(len(s)), key = lambda k : s[k])
    nonzero_sortedlist = []

    for i in range(len(sorted_s)):
        # i to be index of sorted_s with type list
        # sorted_s[i] to be index of distance_matrix[i]
        index = sorted_s[i]
        if distance_matrix[row][index] == 0:
#             print("index %d is %d" %(index, 0))
            continue
        else:
#             print("non-zero index : %d , value = %3.3f" %(index,distance_matrix[0][index]))
            nonzero_sortedlist.append(index)

#     print(sorted_s)
#     print(nonzero_sortedlist)
    return nonzero_sortedlist

In [6]:
'''
    retrieve Nearest Neighbor for each sample of distance matrix
    return nearest_neighbor of each sample contains non-zero distance
    return type : list
'''
def nearest_neighbor(dis_matrix):
    samples = len(dis_matrix)
    NN_list = []
    
    for row in range(samples):
        NN_list.append(sorted_index_per_row(dis_matrix[row], row))
    
#     print(NN_list)
    return NN_list

In [7]:
nearest_neighbor_list = nearest_neighbor(distance_matrix)
nearest_neighbor_list

[[4, 1, 3, 2],
 [0, 4, 2, 5],
 [3, 5, 4, 0, 1],
 [2, 4, 5, 0],
 [0, 3, 5, 1, 2],
 [3, 2, 4, 1]]

In [8]:
def return_mask_array(missing_row, missing_col, k_neighbors):
#     data = data.copy()
    row = missing_row
    col = missing_col
    k = k_neighbors
    '''
        create a mask-array to indicate validity value of potential donors among original data
        if the value of a donor at the same column is NaN, mask[data_idx] = True, False otherwise 
        size of mask array responding to length of nearest_neighbor_list[missing_row]
        example:
            missing_row = 0 
            nearest_neighbor_list[0] = [4, 1, 3, 2]
            initial mask = [None, None, None, None]
            check data[0][4] = 5, mask = [False, None, None, None]
            check data[0][1] = 2, mask = [False, False, None, None] 
            check data[0][3] = NaN, mask = [False, False, True, None]
            check data[0][2] = 8, mask = [False, Fasle, True, False]
    '''
    if k > len(nearest_neighbor_list[row]):
        k = len(nearest_neighbor_list[row])
    elif k <= len(nearest_neighbor_list[row]):
        pass
        
#     mask = [None]*len(nearest_neighbor_list[row])
    mask = [None]*k

    for i in range(k):
        '''
            ridx is each item of nearest_neighbor_list[missing_row]
            missing_row = 0
            nearest_neighbor_list[0] = [4, 1, 3, 2]
                ridx = nearest_neighbor_list[0][0] = 4
                ridx = nearest_neighbor_list[0][1] = 1
                ridx = nearest_neighbor_list[0][2] = 3
                ridx = nearest_neighbor_list[0][3] = 2
        '''
        ridx = nearest_neighbor_list[row][i]
        if np.isnan(data[ridx][col]):
            mask[i] = True
        else:
            mask[i] = False
            
    # print(nearest_neighbor_list[row])
    # print(mask)
    return mask

In [9]:
def imputation(dataset, num_of_neighbor):#, weighted_type):
    missing_data = dataset.copy()
    imputed_data = dataset.copy()
    rows, cols = dataset.shape
    
    ## parameters setting at initial
    k = num_of_neighbor
    #weight_type = weighted_type
    
    for row in range(rows):
        for col in range(cols):
            '''
                check every cell in data is NaN or not
                if so, then do the imputation process
                if not, the skip current iteration
            '''
            if np.isnan(missing_data[row][col]) :
#                 print("missing value at [%d][%d]\n" %(row, col))
                
                nn = nearest_neighbor_list[row][:k]
#                 print(nn)
                
                mask_array = return_mask_array(row, col, k)
                data_instance = []
                
                #check content of mask array
                if all(item == True for item in mask_array):
                    # all contents of a mask array is True
#                     print("original mask array are ALL TRUE as below : ")
#                     print(mask_array)
                    
                    # reset mask array
                    for i in range(len(mask_array)):
                        mask_array[i] = False
#                     print("reset content of mask array : ")
#                     print(mask_array)
                        
                    while(len(data_instance)) < len(mask_array):
                        rand = rd.randint(0, len(data)-1)
                        new = data[rand][col]
                        if (not np.isnan(new)):
                            data_instance.append(new)
#                     print("random data_instance : ")
#                     print(data_instance)
                    
                else:
                    for i in range(len(nn)):
                        r_idex = nn[i]
                        data_instance.append(data[r_idex][col])
#                         print("original data_instance : ")
#                         print(data_instance)

                mx = ma.array(data_instance, mask = mask_array)
                
#                 print("missing value been imputed value : %3.4f" %(mx.mean()))
                imputed_data[row][col] = mx.mean()
            else:
                continue
            
#             print("\n")
#             print("imputed data")
#             print(imputed_data)
#     print(missing_data)
#     print(imputed_data)
    return imputed_data